In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
import pandas as pd
import numpy as np

from scipy.stats import wasserstein_distance, pearsonr, spearmanr

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import string
from nltk import word_tokenize
import nltk
nltk.download('punkt')

from tqdm import tqdm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [46]:
from torchtext import data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [47]:
from torchtext.vocab import FastText
vectors = FastText('simple')

In [48]:
BATCH_SIZE = 128
EPOCHS = 100
LR = 0.0005
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [49]:
train_path = '/content/drive/MyDrive/emotion detection/train/'
test_path = '/content/drive/MyDrive/emotion detection/test/'
train_csv = 'train_df.csv'
test_csv = 'test_df.csv'

In [50]:
train_df = pd.read_csv(train_path + train_csv, index_col='index')
test_df = pd.read_csv(test_path + test_csv, index_col='index')

In [51]:
target_cols = train_df.columns[-6:]
train_df_n = train_df.copy()
train_df_n[target_cols] = train_df_n[target_cols].apply(lambda it: it/it.sum() if it.sum() else 0, axis=1)
train_df_n['target'] = train_df_n[target_cols].apply(lambda it: list(it), axis=1)
test_df_n = test_df.copy()
test_df_n[target_cols] = test_df_n[target_cols].apply(lambda it: it/it.sum() if it.sum() else 0, axis=1)
test_df_n['target'] = test_df_n[target_cols].apply(lambda it: list(it), axis=1)
train_df_n.head()

,text,0,1,2,3,4,5,target
index,,,,,,,,
1,Mortar assault leaves at least 18 dead,0.148649,0.013514,0.405405,0.000000,0.432432,0.000000,"[0.14864864864864866, 0.013513513513513514, 0...."
2,Goal delight for Sheva,0.000000,0.000000,0.000000,0.709924,0.000000,0.290076,"[0.0, 0.0, 0.0, 0.7099236641221374, 0.0, 0.290..."
3,Nigeria hostage feared dead is freed,0.081448,0.000000,0.235294,0.298643,0.090498,0.294118,"[0.08144796380090498, 0.0, 0.23529411764705882..."
4,Bombers kill shoppers,0.231579,0.136842,0.329825,0.000000,0.301754,0.000000,"[0.23157894736842105, 0.1368421052631579, 0.32..."
5,"Vegetables, not fruit, slow brain decline",0.000000,0.000000,0.252525,0.262626,0.020202,0.464646,"[0.0, 0.0, 0.25252525252525254, 0.262626262626..."


In [52]:
train_input = []
train_target = []
test_input = []
test_target = []
# preprocess
for i in train_df_n.index:
  t = train_df_n.loc[i, 'text'].lower()
  t = "".join([char for char in t if char not in string.punctuation])
  train_input.append(word_tokenize(t))
  train_target.append(list(train_df_n.loc[i, target_cols]))

for i in test_df_n.index:
  t = test_df_n.loc[i, 'text'].lower()
  t = "".join([char for char in t if char not in string.punctuation])
  test_input.append(word_tokenize(t))
  test_target.append(list(test_df_n.loc[i, target_cols]))

In [53]:
max_features = 100000

tk = Tokenizer(lower = True, filters='', num_words=max_features)
full_text = train_input + test_input
# full_text = list(test_df_n['text'].values) + list(test_df_n['text'].values)
tk.fit_on_texts(full_text)

In [54]:
word2idx = tk.get_config()['word_index']

In [55]:
word2idx = eval(word2idx)
max(word2idx.values())

3457

In [56]:
def load_glove(path):
    """
    creates a dictionary mapping words to vectors from a file in glove format.
    """
    with open(path) as f:
        glove = {}
        for line in f.readlines():
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            glove[word] = vector
        return glove
def load_glove_embeddings(path, word2idx, embedding_dim=100):
    with open(path) as f:
        embeddings = np.zeros((max(word2idx.values()) + 1, embedding_dim))
        for line in f.readlines():
            values = line.split()
            word = values[0]
            index = word2idx.get(word)
            if index:
                vector = np.array(values[1:], dtype='float32')
                embeddings[index] = vector
        return torch.from_numpy(embeddings).float()

In [57]:
train_tokenized = tk.texts_to_sequences(train_df_n['text'].fillna('missing'))
test_tokenized = tk.texts_to_sequences(test_df_n['text'].fillna('missing'))

max_len = 72
maxlen = 72
X_train = torch.tensor(pad_sequences(train_tokenized, maxlen = max_len), dtype=torch.long)
X_test = torch.tensor(pad_sequences(test_tokenized, maxlen = max_len), dtype=torch.long)

In [58]:
y_train = torch.tensor(train_df_n[target_cols].values, dtype=torch.float32)
y_test = torch.tensor(test_df_n[target_cols].values, dtype=torch.float32)

In [59]:
X_train

tensor([[   0,    0,    0,  ...,  676,  677,   50],
        [   0,    0,    0,  ..., 1244,    3, 1245],
        [   0,    0,    0,  ...,   50,   11,  282],
        ...,
        [   0,    0,    0,  ...,  285,  142,  436],
        [   0,    0,    0,  ...,   48,  180,  174],
        [   0,    0,    0,  ...,   43,   73, 1686]])

In [60]:
X_train.shape, y_train.shape

(torch.Size([250, 72]), torch.Size([250, 6]))

In [61]:
train = torch.utils.data.TensorDataset(X_train, y_train)
test = torch.utils.data.TensorDataset(X_test, y_test)
    
train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=1000, shuffle=False)

In [62]:
# Bi-LSTM(Attention) Parameters
embedding_dim = 100
n_hidden = 100
num_classes = len(train_target[0])
words = set([c for sublist in train_input+test_input for c in sublist])
vocab_size = len(words)
classes = 6

In [100]:
glove_twitter_path = '/content/drive/MyDrive/emotion detection/glove.twitter.27B.100d.txt'
glove_wiki_path = '/content/drive/MyDrive/emotion detection/glove.6B.100d.txt'
glove = load_glove(glove_twitter_path)

In [101]:
toy_embeddings = load_glove_embeddings(glove_wiki_path, word2idx)
toy_embeddings.requires_grad = False

In [102]:
class AttentionModel(torch.nn.Module):
	def __init__(self, weights = None):
		super(AttentionModel, self).__init__()
		"""
		Arguments
		---------
		batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
		output_size : 2 = (pos, neg)
		hidden_sie : Size of the hidden_state of the LSTM
		vocab_size : Size of the vocabulary containing unique words
		embedding_length : Embeddding dimension of GloVe word embeddings
		weights : Pre-trained GloVe word_embeddings which we will use to create our word_embedding look-up table 
		
		--------
		
		"""
		self.batch_size = BATCH_SIZE
		self.output_size = 6
		self.hidden_size = n_hidden
		self.vocab_size = vocab_size
		self.embedding_length = embedding_dim
		self.word_embeddings = nn.Embedding(vocab_size, self.embedding_length).from_pretrained(weights)
		# self.word_embeddings.weights = nn.Parameter(weights, requires_grad=False)
		self.lstm = nn.LSTM(self.embedding_length, self.hidden_size, bidirectional=True, dropout=0.2)
		self.label = nn.Linear(self.hidden_size*2, self.output_size)
		self.dropout1 = nn.Dropout(0)
		self.dropout2 = nn.Dropout(0.5)
 
		
	def attention_net(self, lstm_output, final_state):
		""" 
		Now we will incorporate Attention mechanism in our LSTM model. In this new model, we will use attention to compute soft alignment score corresponding
		between each of the hidden_state and the last hidden_state of the LSTM. We will be using torch.bmm for the batch matrix multiplication.
		
		Arguments
		---------
		
		lstm_output : Final output of the LSTM which contains hidden layer outputs for each sequence.
		final_state : Final time-step hidden state (h_n) of the LSTM
		
		---------
		
		Returns : It performs attention mechanism by first computing weights for each of the sequence present in lstm_output and and then finally computing the
				  new hidden state.
				  
		Tensor Size :
					hidden.size() = (batch_size, hidden_size)
					attn_weights.size() = (batch_size, num_seq)
					soft_attn_weights.size() = (batch_size, num_seq)
					new_hidden_state.size() = (batch_size, hidden_size)
					  
		"""
		hidden = final_state.view(-1, n_hidden*2, 1)
		attn_weights = torch.bmm(lstm_output, hidden).squeeze(2)
		soft_attn_weights = F.softmax(attn_weights, 1)
		new_hidden_state = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)
		
		return new_hidden_state
	
	def forward(self, input_sentences, batch_size=None):
		""" 
		Parameters
		----------
		input_sentence: input_sentence of shape = (batch_size, num_sequences)
		batch_size : default = None. Used only for prediction on a single sentence after training (batch_size = 1)
		
		Returns
		-------
		Output of the linear layer containing logits for pos & neg class which receives its input as the new_hidden_state which is basically the output of the Attention network.
		final_output.shape = (batch_size, output_size)
		
		"""
		input = self.word_embeddings(input_sentences)
		input = input.permute(1, 0, 2)
		if batch_size is None:
			h_0 = Variable(torch.zeros(2, self.batch_size, self.hidden_size))
			c_0 = Variable(torch.zeros(2, self.batch_size, self.hidden_size))
		else:
			h_0 = Variable(torch.zeros(2, batch_size, self.hidden_size))
			c_0 = Variable(torch.zeros(2, batch_size, self.hidden_size))
		output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
		output = output.permute(1, 0, 2)
		attn_output = self.attention_net(self.dropout1(output), final_hidden_state)
		logits = self.label(self.dropout2(attn_output))
		return logits
		# return torch.nn.functional.softmax(logits)

In [103]:
# from torchtext.vocab import FastText
# ft_embedding = FastText('simple')
# weights = torch.FloatTensor(ft_embedding.vectors)

In [104]:
model = AttentionModel(nn.Parameter(toy_embeddings, requires_grad=False))
# model.word_embeddings.weight = nn.Parameter(toy_embeddings, requires_grad=False)
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=0.001)
criterion = nn.MSELoss()

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [105]:
best_param = 0
best_loss = 0
best_metric_param = 0
best_metric = 0
for e in range(EPOCHS):
    for i, (texts, labels) in tqdm(enumerate(train_loader)):
      optimizer.zero_grad()
      pred = model(texts, batch_size=texts.shape[0])
      loss = criterion(pred, labels)
      loss.backward()
      optimizer.step()
    with torch.no_grad():
        for i, (texts, labels) in tqdm(enumerate(test_loader)):
          pred = model(texts, batch_size=1000)
          loss = criterion(pred, labels)
          RMSED = np.mean([np.sqrt(criterion(pred[i], labels[i])) for i in range(1000)])
          if best_param == 0 or (loss.item() < best_loss):
              best_param = model.state_dict()
              best_loss = loss.item()
          if best_metric_param == 0 or (RMSED < best_metric):
              best_metric_param = model.state_dict()
              best_metric = RMSED
          if (e % 2 == 0):
            print(f'epoch {e}: loss: {loss.item()}, RMSED: {RMSED}')

2it [00:00,  3.23it/s]
1it [00:00,  1.04it/s]


epoch 0: loss: 0.06463617086410522, RMSED: 0.2500159442424774


2it [00:00,  2.85it/s]
1it [00:00,  1.03it/s]
2it [00:00,  2.92it/s]
1it [00:01,  1.02s/it]


epoch 2: loss: 0.05602327361702919, RMSED: 0.23151318728923798


2it [00:00,  2.85it/s]
1it [00:01,  1.01s/it]
2it [00:00,  3.29it/s]
1it [00:00,  1.08it/s]


...

2it [00:00,  2.88it/s]
1it [00:00,  1.00it/s]
2it [00:00,  3.07it/s]
1it [00:00,  1.06it/s]


epoch 96: loss: 0.03390655666589737, RMSED: 0.1745503842830658


2it [00:00,  3.40it/s]
1it [00:00,  1.09it/s]
2it [00:00,  3.36it/s]
1it [00:00,  1.06it/s]


epoch 98: loss: 0.034050874412059784, RMSED: 0.17420363426208496


2it [00:00,  3.36it/s]
1it [00:00,  1.08it/s]


In [106]:
best_loss, best_metric

(0.033519282937049866, 0.17346622)

In [107]:
model.load_state_dict(best_metric_param)

<All keys matched successfully>

In [108]:
def acc(x, y):
  return np.argmax(x) == np.argmax(y)

In [109]:
with torch.no_grad():
  for _, (texts, labels) in tqdm(enumerate(test_loader)):
    pred = model(texts, batch_size=1000)
    loss = criterion(pred, labels)
    acc_1 =  np.mean([acc(pred[i], labels[i]) for i in range(1000)])
    APd = np.mean([pearsonr(pred[i], labels[i]) for i in range(1000)])
    ASd = np.mean([spearmanr(pred[i], labels[i]) for i in range(1000)])
    APe = np.mean([pearsonr(pred[:, i], labels[:, i]) for i in range(6)])
    RMSED = np.mean([np.sqrt(criterion(pred[i], labels[i])) for i in range(1000)])
    WD = np.mean([wasserstein_distance(pred[i], labels[i]) for i in range(1000)])

1it [00:02,  2.06s/it]


In [110]:
loss.item(), acc_1, APd, ASd, APe, RMSED, WD

(0.03349388763308525,
 0.414,
 0.378681727369863,
 0.377293546274097,
 0.15366352624215018,
 0.17295001,
 0.10663217979374652)

In [112]:
print("------\nAcc@1: ", acc_1, "\nAPd: ", APd, "\nAPe: ",APe, "\nRMSED: ",RMSED, "\nWDD: ",WD)
'------'

------
Acc@1:  0.414 
APd:  0.378681727369863 
APe:  0.15366352624215018 
RMSED:  0.17295001 
WDD:  0.10663217979374652


'------'